In [ ]:
## Install the gemini API
!pip install -q -U google-generativeai

In [18]:
def step_one_cleaning(prompt):  
    from io import StringIO
    import csv
    import pathlib
    import textwrap
    import google.generativeai as genai
    import numpy as np
    import pandas as pd
    
    ## Need to change this line to get your own API key if possible
    from config import GOOGLE_API_KEY
    
    
    #setting the model and API key
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    
    # Input string prompt to the google model 
    response = model.generate_content(prompt)
    
    # Define the data string
    data_string = response.text
    
    # Create a StringIO object to mimic a file-like object
    data_io = StringIO(data_string)
    
    # Create first df to be cleaned and returned 
    df = pd.read_csv(data_io, delimiter="|")
    
    # Drop empty top row 
    df = df.iloc[1:].reset_index(drop = True)
    
    # Drop empty columns
    names = df.columns
    for name in names: 
        if "unnamed" in name.lower(): 
            df = df.drop(columns = [name])
    return df

In [19]:
def step_two_cleaning(prompt):
    import numpy as np
    import pandas as pd
    status = "broken"
    while status == "broken":
        df = step_one_cleaning(prompt)
        names = df.columns
        if len(names) == 3 and len(df) != 0:
            props = np.array(df[names[2]])
            for i in range(len(props)): 
                if df[names[2]][i] == "**1**":
                     df[names[2]][i] = "1.00"
            status = "clean"
    # Making proportion column a float between 0 and 1
    names = df.columns
    print(names)
    names_2 = ["version_4", "version_3.1", "Proportion of Jobs"]
    df.columns = names_2
    ## this is to ensure that the end of the data set is not a row that says "Total .... 1.00" which the prompt often returns
    ## This needs to be robust if the column type is string or float/int
    
    if type(df["version_4"].iloc[-1]) == str:
        if "total" in df["version_4"].iloc[-1].lower() or df["version_4"].iloc[-1].lower() == '': 
            df = df.iloc[:-1]
    elif type(df["version_3.1"].iloc[-1]) == str:
        if "total" in df["version_3.1"].iloc[-1].lower():
            df = df.iloc[:-1]
    
    # Making proportion column a float between 0 and 1
    if type(df["Proportion of Jobs"][0]) == str:
        df["Proportion of Jobs"] = df["Proportion of Jobs"].str.strip()
        df["Proportion of Jobs"] = df["Proportion of Jobs"].str.replace("%", "")
        df["Proportion of Jobs"] = df["Proportion of Jobs"].astype(float) 
    if df["Proportion of Jobs"][0] > 1:
        df["Proportion of Jobs"] = df["Proportion of Jobs"]/100 
    
    # Replace non-numeric characters with an empty string
    pattern = r'\D'  # Matches any non-digit character
    df["version_4"] = df["version_4"].str.replace(pattern, '')    
    df["version_3.1"] = df["version_3.1"].str.replace(pattern, '')
    
    ## scale the data frame
    df = df.reset_index(drop = True)
    if df["Proportion of Jobs"].sum() > 1:
        total = df["Proportion of Jobs"].sum()
        for i in range(len(df["Proportion of Jobs"])):
            df["Proportion of Jobs"][i] = df["Proportion of Jobs"][i]/total
    return df

In [4]:
##Function takes the created prompt and returns a dataframe that has proportions showing how each 3 digit cade in ISIC 4 is created in ISIC 3.1

def get_proportions(prompt):
    from io import StringIO
    import csv
    import pathlib
    import textwrap
    import google.generativeai as genai
    import numpy as np
    import pandas as pd
    
    ## Need to change this line to get your own API key if possible
    from config import GOOGLE_API_KEY
    
    
    #setting the model and API key
    genai.configure(api_key=GOOGLE_API_KEY)
    model = genai.GenerativeModel('gemini-pro')
    
    # Input string prompt to the google model 
    response = model.generate_content(prompt)
    
    # Define the data string
    data_string = response.text
    
    # Create a StringIO object to mimic a file-like object
    data_io = StringIO(data_string)
    
    # Create first df to be cleaned and returned 
    df = pd.read_csv(data_io, delimiter="|")
    
    # Drop empty top row 
    df = df.iloc[1:].reset_index(drop = True)
    
    # Drop empty columns
    names = df.columns
    for name in names: 
        if "Unnamed" in name: 
            df = df.drop(columns = [name])
    
    # Making proportion column a float between 0 and 1
    names = df.columns
    names_2 = ["version_4", "version_3.1", "Proportion of Jobs"]
    df.columns = names_2
    ## this is to ensure that the end of the data set is not a row that says "Total .... 1.00" which the prompt often returns
    ## This needs to be robust if the column type is string or float/int
    if type(df["Proportion of Jobs"].iloc[-1]) == str:
        if "total" in df["version_4"].iloc[-1].lower():
            df = df.iloc[:-1]
    else:
        if df["Proportion of Jobs"].iloc[-1] == 1:
            df = df.iloc[:-1]
    
     # Making proportion column a float between 0 and 1
    df["Proportion of Jobs"] = df["Proportion of Jobs"].str.strip()
    df["Proportion of Jobs"] = df["Proportion of Jobs"].str.replace("%", "")
    df["Proportion of Jobs"] = df["Proportion of Jobs"].astype(float) 
    if df["Proportion of Jobs"][0] > 1:
        df["Proportion of Jobs"] = df["Proportion of Jobs"]/100 
    
    # Replace non-numeric characters with an empty string
    pattern = r'\D'  # Matches any non-digit character
    df["version_4"] = df["version_4"].str.replace(pattern, '')    
    df["version_3.1"] = df["version_3.1"].str.replace(pattern, '')
    
    ## scale the data frame
    df = df.reset_index(drop = True)
    if df["Proportion of Jobs"].sum() > 1:
        total = df["Proportion of Jobs"].sum()
        for i in range(len(df["Proportion of Jobs"])):
            df["Proportion of Jobs"][i] = df["Proportion of Jobs"][i]/total
    return df

In [11]:
def make_prompt(digits, code):
    if digits == "three":
        num_codes = corr_table["ISIC_4_3d"].value_counts()
        prompt = "A " + digits + " digit code "+ code + " which is (" + str(ISIC_4_3digit[ISIC_4_3digit["code"] == code]["description"].iloc[0]) + ") in ISIC version 4 is comprised of " + str(num_codes[code]) + " 4 digit codes in ISICs version 3.1, "
        codes_31 = corr_table[corr_table["ISIC_4_3d"] == code]
        ## Code to handle when there are multiple instances of a code and possibly multiple "details" for that code in the correspondence table
        for code in codes_31["ISIC31code"].unique(): 
            ## start by just adding the code and its standard description from the ISIC code data frame
            prompt = prompt + code + " which is (" + ISIC_3_4digit[ISIC_3_4digit['code'] == code]['description'].iloc[0] + "), "
            ## now test if the code is unique, that is, it only appears once within the 3 digit code we are considering from version 4
            if codes_31['ISIC31code'].eq(code).sum() == 1:
                ## If it is unique, we test if the detail column of the correspondence table is empty, if it is not we add that extra detail to the prompt
                if codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[0] != "":
                    prompt = prompt + " and includes (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[0] + ") "
            elif codes_31['ISIC31code'].eq(code).sum() > 1:
                ## we now test if the code is not unique, if this is the case our code is more complicated, start by making an index list that will track appearances of this code
                ind = []
                ## Now we will loop through the instances of the code and add the index of each instance if the detail section is not empty, we are collecting all the different details to include 
                for i in range(codes_31['ISIC31code'].eq(code).sum()):
                    if codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[i] != "":
                        ind.append(i)
                ## If the code is repeated but never with a detail the prompt remains unchanged
                if len(ind) == 0:
                    prompt = prompt
                ## If the code is repeated but only with 1 detail, just include that single detail
                elif len(ind) == 1:
                    prompt = prompt + "and includes (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[ind[0]] + ", "
                ## If the code is repeated with multiple details, start with the first detail and drop the first item in the list so it is not repeated, then loop through the list adding all the details
                else: 
                    prompt = prompt + " and includes (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[ind[0]]
                    ind = ind[1:]
                    for k in ind:
                        prompt = prompt + ", and (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[k] + ")"
        if num_codes[three_code] > 10:
            prompt = prompt + " What is your best estimate of the proportion of jobs now coded in " + code + " that were in each of the previous codes in version 3.1? The proportions can be less than .05 and most probably are. Can you give me your best guesses in a table with 3 columns, first the three digit code, " + code + " then the four didgit codes, then the proportions?"
        else: 
            prompt = prompt + " What is your best estimate of the proportion of jobs now coded in " + code + " that were in each of the previous codes in version 3.1? Can you give me your best guesses in a table with 3 columns, first the three digit code, " + code + " then the four didgit codes, then the proportions?"
    if digits == "four":
        num_codes = corr_table["ISIC4code"].value_counts()
        prompt = "A " + digits + " digit code " + code + " which is (" + str(ISIC_4_4digit[ISIC_4_4digit["code"] == code]["description"].iloc[0]) + ") in ISIC version 4 is comprised of " + str(num_codes[code]) + " four digit codes in ISICs version 3.1, "
        codes_31 = corr_table[corr_table["ISIC4code"] == code]
        ## Code to handle when there are multiple instances of a code and possibly multiple "details" for that code in the correspondence table
        for code in codes_31["ISIC31code"].unique(): 
            ## start by just adding the code and its standard description from the ISIC code data frame
            prompt = prompt + code + " which is (" + ISIC_3_4digit[ISIC_3_4digit['code'] == code]['description'].iloc[0] + "), "
            ## now test if the code is unique, that is, it only appears once within the 3 digit code we are considering from version 4
            if codes_31['ISIC31code'].eq(code).sum() == 1:
                ## If it is unique, we test if the detail column of the correspondence table is empty, if it is not we add that extra detail to the prompt
                if codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[0] != "":
                    prompt = prompt + " and includes (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[0] + ") "
            elif codes_31['ISIC31code'].eq(code).sum() > 1:
                ## we now test if the code is not unique, if this is the case our code is more complicated, start by making an index list that will track appearances of this code
                ind = []
                ## Now we will loop through the instances of the code and add the index of each instance if the detail section is not empty, we are collecting all the different details to include 
                for i in range(codes_31['ISIC31code'].eq(code).sum()):
                    if codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[i] != "":
                        ind.append(i)
                ## If the code is repeated but never with a detail the prompt remains unchanged
                if len(ind) == 0:
                    prompt = prompt
                ## If the code is repeated but only with 1 detail, just include that single detail
                elif len(ind) == 1:
                    prompt = prompt + "and includes (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[ind[0]] + ", "
                ## If the code is repeated with multiple details, start with the first detail and drop the first item in the list so it is not repeated, then loop through the list adding all the details
                else: 
                    prompt = prompt + " and includes (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[ind[0]]
                    ind = ind[1:]
                    for k in ind:
                        prompt = prompt + ", and (" + codes_31[codes_31["ISIC31code"] == code]["Detail"].iloc[k] + ")"
        if num_codes[code] > 10:
            prompt = prompt + " What is your best estimate of the proportion of jobs now coded in " + code + " that were in each of the previous codes in version 3.1? The proportions can be less than .05 and many probably are less .05. Can you make sure these proportions all sum to 1? Can you give me your best guesses in a table with 3 columns, first the three digit code, " + code + " then the four didgit codes, then the proportions?"
        else: 
            prompt = prompt + " What is your best estimate of the proportion of jobs now coded in " + code + " that were in each of the previous codes in version 3.1? If there is only 1 version 3.1 code, it is automatically 1. Can you give me your best guesses in a table with 3 columns, first the version 4 code, " + code + " then the version 3.1 codes, then the proportions?"
    else:
        error_message = "Digits must be specified either to be three or four, more functionality coming soon, thank you!"
        return error_message
    return prompt

In [14]:
## Creating a usable prompt 
## Start by importing the correspondence table and .txt files with with codes and descriptions
## We scraped the PDFs of the ISIC codes
import pandas as pd
import numpy as np
corr_table = pd.read_csv("ISIC4_ISIC31.csv", dtype={"ISIC4code": str, "partialISIC4": str, "ISIC31code": str, "partialISIC31": str})
corr_table = corr_table.fillna("")
ISIC_4 = pd.read_csv("isic4.txt", dtype={'num': str}, delimiter = "|")
ISIC_31 = pd.read_csv("isic31.txt", dtype={'num': str}, delimiter = "|")
col_names = ["code", "description"]
ISIC_4.columns = col_names
ISIC_31.columns = col_names

In [7]:
# Cleaning data and making dfs for each code length 
code_length_4 = ISIC_4['code'].str.len()
code_length_31 = ISIC_31['code'].str.len()

# Filter DataFrames based on code length
ISIC_4_2digit = ISIC_4[code_length_4 == 2]
ISIC_4_3digit = ISIC_4[code_length_4 == 3]
ISIC_4_4digit = ISIC_4[code_length_4 == 4]

ISIC_3_2digit = ISIC_31[code_length_31 == 2]
ISIC_3_3digit = ISIC_31[code_length_31 == 3]
ISIC_3_4digit = ISIC_31[code_length_31 == 4]

# Making 3 digit and 2 digit 4.0 codes columns in the correspondence table 
corr_table['ISIC_4_3d'] = corr_table['ISIC4code'].str[:3]
corr_table['ISIC_4_2d'] = corr_table['ISIC4code'].str[:2]

In [28]:
codes = np.array(ISIC_4_4digit["code"])[:60]
df_big = pd.DataFrame({'version_4': [], 'version_3.1': [], "Proportion of Jobs": []})
i = 0
for code in codes:
    if code == "0200":
        continue
    prompt = make_prompt("four", code)
    df = step_two_cleaning(prompt)
    i += 1
    print(i)
    df_big = pd.concat([df_big, df])
    df_big = df_big[df_big.notna().all(axis=1)]
    df_big = df_big.reset_index(drop = True)
df_big.to_excel("correspondence_table.xlsx", index=True)
print(df_big)

Index([' Version 4 Code ', ' Version 3.1 Codes ', ' Proportions '], dtype='object')
1
Index([' Version 4 code ', ' Version 3.1 code ', ' Proportion '], dtype='object')
2
Index([' ISIC v4 Code ', ' ISIC v3.1 Codes ', ' Proportion '], dtype='object')
3
Index([' ISIC Version 4 Code ', ' ISIC Version 3.1 Codes ', ' Proportion '], dtype='object')
4
Index([' ISIC4 ', ' ISIC3.1 ', ' Proportion '], dtype='object')
5
Index([' ISIC V4 Code ', ' ISIC V3.1 Code ', ' Proportion '], dtype='object')
6
Index([' ISIC v4 code ', ' ISIC v3.1 codes ', ' Proportion '], dtype='object')
7
Index([' ISIC 4 Code ', ' ISIC 3.1 Code ', ' Proportion '], dtype='object')
8
Index([' Version 4 Code ', ' Version 3.1 Codes ', ' Proportion '], dtype='object')
9
Index([' Version 4 Code ', ' Version 3.1 Code ', ' Proportion '], dtype='object')
10
Index([' ISIC v4 code ', ' ISIC v3.1 code ', ' Proportion '], dtype='object')
11
Index([' ISIC 4 code ', ' ISIC 3.1 code ', ' Proportion '], dtype='object')
12
Index([' Version 4 

KeyError: '0200'

In [30]:
len(np.array(ISIC_4_4digit["code"]))

413